## Recommendation Engine

Author: Oleg Zarakhani

Version Date: 2023-03-15

This notebook will explore how to use historical user purchase data in order to create a recommendation model which will attempt to guess which products out of a basket of items the customer will be likely to purchase at a given point in time. 

At DataRobot, Multilabel modelling is a kind of classification task that, while similar to multiclass modelling, provides more flexibility. In multi-label modelling, each row in a dataset is associated with one or several labels.
Extending this framework in our unlimited label mode and paired with feature discovery, allow the user frame a model that can be used to serve recommendations. Given the use-case, this recommendation model can provide rank ordered suggestions of content, product, or services that a specific customer might like.

As an example, if we have historic purchases of a sample of customers, we can look at common spending habits across demographics and shopping baskets, identify new features and able to rank order anticipated items at the customer level. Some of the features automatically generated might be most common category of item per a specific geography, or the degree of a customers proclivity to try new things.

In [7]:
import pandas as pd
import numpy as np
import json

### Load in relevant datasets

### Products

This dataset contains the products, inclusing the product details that made up all the orders. The total amount of orders in the dataset is 22064 and as an example, below is what a products from a couple of orders look like:

In [9]:
order_products.order_id = order_products.order_id.astype(int)
print('total_number of orders: {}'.format(order_products.order_id.nunique()))
order_products[order_products.order_id.isin([518, 4418.0])].sort_values('order_id')

total_number of orders: 22064


,order_id,reordered,order_product_id,product_name,aisle,department
94991,518,1,518_1,Organic Avocado,fresh fruits,produce
84002,518,1,518_5,Bag of Organic Bananas,fresh fruits,produce
137477,518,1,518_3,Smoked Fresh Turkey Kielbasa,hot dogs bacon sausage,meat seafood
43512,518,0,518_7,Carrots,fresh vegetables,produce
203134,518,1,518_2,Organic Extra Firm Tofu,tofu meat alternatives,deli
59407,518,1,518_4,Organic Zucchini,fresh vegetables,produce
136005,518,0,518_6,Beef Short Ribs,meat counter,meat seafood
57576,518,1,518_8,Russet Potato,fresh vegetables,produce
141401,4418,1,4418_12,Gluten Free Millet-Chia Bread,bread,bakery
145245,4418,0,4418_16,Gluten free Sesame Bagels,breakfast bakery,bakery


### Orders

This dataset contains the orders along with the order time and the corresponding user_id of who-ever placed the order. Below is a sample of orders from March 1st in this example dataset.

In [10]:
orders.loc[orders.order_time.dt.strftime('%Y-%m-%d') == '2015-03-01'].head(10)

,order_id,user_id,order_time
53,3000001,180,2015-03-01 07:00:00
178,1672206,311,2015-03-01 22:00:00
260,1785222,606,2015-03-01 04:00:00
435,822880,800,2015-03-01 13:00:00
591,359937,1169,2015-03-01 00:00:00
619,444002,1199,2015-03-01 02:00:00
949,2396674,1866,2015-03-01 02:00:00
1692,228107,3190,2015-03-01 18:00:00
1796,1036072,3369,2015-03-01 11:00:00
1963,1428014,3659,2015-03-01 19:00:00


## Primary Training Data

### Most common products

The next thing that we’ll need to do is create the `train` dataset. The target for each row in the `train` dataset will contain which of the top most common global items that this customer has purchased on a specific date. This list of the top most common global items will be the pool of the items to recommend. Rationale here is that rare items that are purchased rarely will be difficult to recommend so we will omit them from modelling. Below is a calculation to calculate the most common items purchased from the `order_products` dataset

In [11]:
order_counts = pd.DataFrame(order_products.groupby('product_name').size(), columns=['count'])\
                                .reset_index()\
                                .sort_values('count', ascending=False)\
                                .reset_index()
order_counts.head(10)

,index,product_name,count
0,1201,Banana,5124
1,1118,Bag of Organic Bananas,3975
2,11241,Organic Strawberries,2202
3,9553,Organic Baby Spinach,1957
4,10348,Organic Hass Avocado,1842
5,9535,Organic Avocado,1529
6,15039,Strawberries,1130
7,11596,Organic Yellow Onion,1088
8,11612,Organic Zucchini,1082
9,7408,Large Lemon,1066


For this example we will set the max number of items to 500. In a multilabel framework the maximum distinct number of classes DataRobot can track is 1000.

In [12]:
max_number_of_items = 500
frequent_orders = order_counts.product_name.head(max_number_of_items).to_list()

print(frequent_orders[0:9])

['Banana', 'Bag of Organic Bananas', 'Organic Strawberries', 'Organic Baby Spinach', 'Organic Hass Avocado', 'Organic Avocado', 'Strawberries', 'Organic Yellow Onion', 'Organic Zucchini']


### DataRobot MultiLabel

Multilabel modeling is a kind of classification task that, while similar to multiclass modeling, provides more flexibility. In multilabel modeling, each row in a dataset is associated with one, several, or zero labels. One common multilabel classification problem is text categorization (e.g., a movie description can include both “Crime”, “Drama” and “Black and White”).


More information: https://docs.datarobot.com/en/docs/modeling/special-workflows/multilabel.html#multilabel-modeling


In this case we will use this framework for the model to pull out the relevant most common items in each basket for the customer and use this as a framework. So if a person baught Bananas, Strawberries and garbage bags, the target variale for that user would be `[‘Banana’, ‘Strawberries’]`.

In [ ]:
order_collector = pd.DataFrame()
for i, row in orders.iterrows():
    order_id = row.order_id
    order_data = order_products.loc[order_products.order_id == order_id]
    items = order_data.loc[order_data.product_name.isin(frequent_orders)]['product_name'].to_list()
#     if len(items)> 3:
#         break
    row['items'] = json.dumps(items)
    order_collector = order_collector.append(row)
multi_label_product_training = order_collector.drop('order_id', axis=1)
multi_label_product_training.order_time = multi_label_product_training.order_time.dt.date
multi_label_product_training.user_id = multi_label_product_training.user_id.astype(int)

In [14]:
multi_label_product_training.head(10)

,user_id,order_time,items
0,79,2015-01-01,"[""Organic Extra Large Grade AA Brown Eggs"", ""O..."
1,79,2015-01-09,"[""Organic Extra Large Grade AA Brown Eggs"", ""O..."
2,79,2015-01-17,"[""Organic Extra Large Grade AA Brown Eggs"", ""W..."
3,79,2015-01-31,"[""Organic Greek Plain Nonfat Yogurt"", ""Organic..."
4,79,2015-02-21,"[""Organic Extra Large Grade AA Brown Eggs"", ""O..."
5,79,2015-03-24,"[""Organic Greek Plain Nonfat Yogurt"", ""Shredde..."
6,79,2015-04-23,"[""Organic Extra Large Grade AA Brown Eggs"", ""T..."
7,79,2015-05-24,[]
8,80,2015-01-01,"[""Bag of Organic Bananas""]"
9,80,2015-01-22,"[""Hass Avocado"", ""Seedless Red Grapes"", ""Bag o..."


### DataRobot API


In order to pass all of the datasets to DataRobot, we will use the feature discovery technique to allow DataRobot to automatically generate the necessary features for us. For more information about Feature Discovery please refer to:<br><br>
**Mastering many tables in production ML Accelerator:** https://community.datarobot.com/t5/ai-accelerators/mastering-many-tables-in-production-ml-complete-workflow/td-p/15905 <br><br>
**Feature Discovery Documentation:** https://docs.datarobot.com/en/docs/data/transform-data/feature-discovery/index.html#feature-discovery

In [15]:
import datarobot as dr
from datarobot.utils import dataframe_to_buffer

dr.Client()

### Upload Datasets to AI Catalog

In [16]:
buff = dataframe_to_buffer(multi_label_product_training)
buff.name = 'multi_label_product_training_dr'
multi_label_product_training_dr = dr.Dataset.create_from_file(filelike=buff)

buff = dataframe_to_buffer(order_products)
buff.name = 'order_products_dr'
order_products_dr = dr.Dataset.create_from_file(filelike=buff)

buff = dataframe_to_buffer(orders)
buff.name = 'orders_dr'
orders_dr = dr.Dataset.create_from_file(filelike=buff)


### Define Relationships

In [17]:
dataset_definitions = [
{
    'identifier': 'orders_dr',
    'catalogVersionId': orders_dr.version_id,
    'catalogId': orders_dr.id,
    'snapshotPolicy': 'latest',
    'primary_temporal_key': 'order_time'
},
{
    'identifier': 'order_products_dr',
    'catalogVersionId': order_products_dr.version_id,
    'catalogId': order_products_dr.id,
    'snapshotPolicy': 'latest',
}]

In [18]:
relationships = [{
        'dataset2Identifier': 'orders_dr',
        'dataset1Keys': ['user_id'],
        'dataset2Keys': ['user_id'],
        'feature_derivation_window_start': -31,
        'feature_derivation_window_end': -1,
        'feature_derivation_window_time_unit': 'DAY',
        'prediction_point_rounding': 1,
        'prediction_point_rounding_time_unit': 'MINUTE'
    },
{
        'dataset1Identifier': 'orders_dr',
        'dataset2Identifier': 'order_products_dr',
        'dataset1Keys': ['order_id'],
        'dataset2Keys': ['order_id'],
    }]


relationship_config = dr.RelationshipsConfiguration.create(dataset_definitions=dataset_definitions, 
                                                    relationships=relationships)

### Start Project

In [ ]:
project = dr.Project.create_from_dataset(multi_label_product_training_dr.id, 
                                         project_name='AI Accelerator - Recommendation Engine')

project.analyze_and_model(
            target='items', 
            relationships_configuration_id=relationship_config.id,
            partitioning_method=dr.GroupCV(holdout_pct=20, reps=5, partition_key_cols=['user_id']),
            # metric='MAPE',
            feature_engineering_prediction_point='order_time',
            mode=dr.enums.AUTOPILOT_MODE.COMPREHENSIVE,
            max_wait = 36000,
            worker_count=-1)

In [ ]:
project.wait_for_autopilot()

## Predictions

Once the model is ready we can pass the predictions file through the model to get a feel for how the the predictions look.

For every puchase DataRobot will output the class probabilities for each of the 500 most common products.

From here we can take the top n probabilities and suggestions for the recommendations. In this example we will look at the top 3 products to recommend for an incoming user.

In [23]:
predict_dataset = test.sample(10)

In [25]:
model = dr.ModelRecommendation.get(
    project.id,
    dr.enums.RECOMMENDED_MODEL_TYPE.RECOMMENDED_FOR_DEPLOYMENT
).get_model()


dataset = project.upload_dataset(predict_dataset)

pred_job = model.request_predictions(dataset.id)
preds = pred_job.get_result_when_complete()


In [26]:
preds = preds[[c for c in preds.columns if 'class' in c]]
preds.columns = [c.replace('class_', '') for c in preds.columns]

In [27]:

top_recommendations = pd.DataFrame(preds.apply(lambda x:list(preds.columns[np.array(x).argsort()[::-1][:3]]), axis=1).to_list(),  
                    columns=['recommendation 1', 'recommendation 2', 'recommendation 3'])

output = pd.concat([predict_dataset[['user_id', 'order_time']].reset_index(drop='True'), top_recommendations], axis=1)

Below we see two datasets. The first one, `predict_dataset` that is the one that we used to pass to the DataRobot Model. The second one, `output` contains the original order times for each user id, along with the associated recommendations for each of those customers.

In [28]:
predict_dataset

,user_id,order_time
895,36244,2015-03-15
315,12723,2015-03-15
227,9483,2015-03-15
313,12655,2015-03-15
378,15090,2015-03-15
686,26948,2015-03-15
624,24362,2015-03-15
539,21509,2015-03-15
292,12010,2015-03-15
659,25685,2015-03-15


In [29]:
output

,user_id,order_time,recommendation 1,recommendation 2,recommendation 3
0,36244,2015-03-15,Organic Fat Free Milk,Organic White Onions,Mixed Berries Whole Milk Icelandic Style Skyr ...
1,12723,2015-03-15,Banana,Organic Hass Avocado,Honeycrisp Apple
2,9483,2015-03-15,Dha Omega 3 Vitamin D Milk,Bag of Organic Bananas,Organic Baby Spinach
3,12655,2015-03-15,Bag of Organic Bananas,Organic Zucchini,Organic Tomato Cluster
4,15090,2015-03-15,Organic Mexican Blend Finely Shredded Cheese,Organic Broccoli Florets,Organic Cucumber
5,26948,2015-03-15,"Yogurt, Strained Low-Fat, Coconut",Organic Hass Avocado,Bag of Organic Bananas
6,24362,2015-03-15,Organic American Cheese Singles,Salted Butter,Organic Small Bunch Celery
7,21509,2015-03-15,Air Chilled Organic Boneless Skinless Chicken ...,Banana,YoKids Blueberry & Strawberry/Vanilla Yogurt
8,12010,2015-03-15,Original Popcorn,Banana,Soda
9,25685,2015-03-15,Banana,Boneless Skinless Chicken Breasts,Organic Lacinato (Dinosaur) Kale
